In [14]:
import sqlite3 as sqlite
from pprint import pprint
TABLE_NAME = "Test"
SRC_DB = "/content/test.db"

def ExecCmd ( msg :str , cmd :str ):
    try:
        conn = sqlite.connect(SRC_DB)
        cur = conn.cursor()
        cur.execute(cmd)
        res = cur.fetchone()
        conn.commit()
        conn.close()
        return res
    except Exception as e:
        print(msg, e)

def NewTable ( ):
    cmd =   f"""CREATE TABLE "{TABLE_NAME}" (
                "KEY"	TEXT NOT NULL UNIQUE,
                "VALUE"	TEXT NOT NULL,
                PRIMARY KEY("KEY"))"""
    ExecCmd("New Table :",cmd)

def InsertOrUpdate ( key :str, value :str ):
    cmd = f"""INSERT OR REPLACE INTO {TABLE_NAME} (KEY,VALUE) values("{key}","{value}") """
    ExecCmd("InsertOrUpdate :", cmd)

def GetByKey ( key :str ) -> str:
    cmd = f"""SELECT KEY,VALUE FROM {TABLE_NAME} WHERE KEY IS "{key}" """
    res = ExecCmd("GetByKey :", cmd)
    return "EMPTY" if res is None else res[1]

def CLEAR_TABLE ():
    cmd = f"""DELETE FROM {TABLE_NAME}"""
    ExecCmd("CLEAR_TABLE :", cmd)


In [7]:
# InsertOrUpdate("eeeee","2222")
res = GetByKey("eeeee")
print(res)
CLEAR_TABLE()

EMPTY


In [8]:
def PUT(key :str, value :str) -> None:
    InsertOrUpdate(key,value)

def GET( key :str ) -> str:
    return GetByKey(key)

def SCAN ( key1 :str, key2 :str ) -> list:
    res = []
    for key in range(int(key1), int(key2)+1):
        res.append( GetByKey(key) )
    return res

def ParseCmd ( cmd :str ) -> list:
    return cmd.split(' ')

In [12]:
def sequential_do_cmd ( input_file :str ) -> list:
    output = []
    with open(input_file,'r') as fin:
        for line in fin:
            cmd = line.replace("\n","")
            res = ParseCmd(cmd)
            if cmd[0] == 'P':
                PUT(res[1],res[2])
            elif cmd[0] == 'G':
                output.append( GET(res[1]) )
            elif cmd[0] == 'S':
                output += SCAN(res[1], res[2])
    return output

In [18]:
outputs = sequential_do_cmd("/content/hw3example.input")

with open("test.output", "w") as fout:
    for output in outputs[:-1]:
        fout.write(f"{output}\n") 
    fout.write(outputs[-1])
    CLEAR_TABLE()